In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime as dt
import datetime, pytz
from sklearn.neighbors import KNeighborsClassifier
import pickle

from ta.trend import MACD
from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator
from ta.volatility import BollingerBands
from ta.volume import AccDistIndexIndicator
from ta.volatility import AverageTrueRange

/var/folders/vh/jb52psn93fb96vlsr5_kn4m80000gn/T/ipykernel_29438/1750041394.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [114]:
# Parameters

leverage = 10
acc_size = 50000
lot_size = 100000

# Connect MT5
(Not available for Mac/Linux)

# Load model

In [115]:
knns = list()
for i in range(100):
    with open(f'models/knn_{i}.pkl', 'rb') as f:
        knn = pickle.load(f)
        knns.append(knn)

/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Data feed

In [134]:
symbols = ['EURUSD=X','JPY=X','GBPUSD=X','CHF=X','AUDUSD=X','CAD=X','NZDUSD=X']

data = dict()
for symbol in symbols:
    data[symbol] = yf.download(symbol,interval='1d',repair=True)

/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/yfinance/base.py:291: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  quotes2 = quotes.resample('30T')
/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/yfinance/base.py:291: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  quotes2 = quotes.resample('30T')
[*********************100%%**********************]  1 of 1 completed
/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and 

In [144]:
def add_ta_signals(df):
    df = df.copy()

    o,h,l,c,v = df['Open'],df['High'],df['Low'],df['Close'],df['Volume']
    
    df['Ret'] = np.log(c).diff().shift(-1)

    df['MACDD'] = MACD(c).macd_diff()
    df['MACDS'] = MACD(c).macd_signal()

    df['RSI'] = RSIIndicator(c).rsi()

    df['SO'] = StochasticOscillator(h,l,c).stoch()
    df['SOS'] = StochasticOscillator(h,l,c).stoch_signal()

    df['BBH'] = BollingerBands(c).bollinger_hband_indicator() - c
    df['BBL'] = BollingerBands(c).bollinger_lband_indicator() - c

    df['ADI'] = AccDistIndexIndicator(h,l,c,v).acc_dist_index()

    df['HLR'] = c / (h.rolling(252).max() - l.rolling(252).min()) - 1 / 2

    df['ATR'] = AverageTrueRange(h,l,c).average_true_range()

    df = df.drop(columns=['Open','High','Low','Close','Adj Close','Volume'])

    try: df.drop(columns=['Repaired?'],inplace=True)
    except: pass

    #df.iloc[:,1:-1] = df.iloc[:,1:-1].div(df.loc[:,'ATR'],axis=0)
    return df

In [145]:
ta_features = dict()
for symbol in symbols:
    ta_data = add_ta_signals(data[symbol])
    ta_features[symbol] = ta_data.iloc[-252:]

# Produce signals

In [147]:
for symbol in symbols:
    features = ta_features[symbol]
    pos = pd.DataFrame()
    for i,knn in enumerate(knns):
        pos[f'{i}'] = knn.predict(features.iloc[:,1:])
    features['Pred'] = pos.mean(axis=1).to_numpy()
    features['Pos'] = (np.around((features['Pred'].expanding().rank(pct=True)*2-1)*5))/5

/var/folders/vh/jb52psn93fb96vlsr5_kn4m80000gn/T/ipykernel_26201/659874341.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Pred'] = pos.mean(axis=1).to_numpy()
/var/folders/vh/jb52psn93fb96vlsr5_kn4m80000gn/T/ipykernel_26201/659874341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Pos'] = (np.around((features['Pred'].expanding().rank(pct=True)*2-1)*5))/5


# Calculate # lot

In [148]:
base_to_usd = { 'EURUSD=X': 'EUR=X',
                'GBPUSD=X': 'GBP=X',
                'AUDUSD=X': 'AUD=X',
                'NZDUSD=X': 'NZD=X'}
prc_to_usd = yf.download(list(base_to_usd.values()))['Close']

/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/benkan45d6/opt/anaconda3/envs/py10/lib/python3.10

In [149]:
utc_datetime = datetime.datetime.now(pytz.utc)
est_datetime = datetime.datetime.now(pytz.timezone('US/Eastern'))
print(utc_datetime)
print(est_datetime)
print('Using the nearest-day position at 20:00 HKT (00:00 UTC)')
print('Open the position at 19:55 since data will disappear at the following 2 hours')
print('Backtest is buying at today\'s close based on data generated at today\'s close, and predicting the return from today to tomorrow')
print('Trading 18x is optimal but 5x is conservative')

2024-01-25 10:29:41.858531+00:00
2024-01-25 05:29:41.858715-05:00
Using the nearest-day position at 20:00 HKT (00:00 UTC)
Open the position at 19:55 since data will disappear at the following 2 hours
Backtest is buying at today's close based on data generated at today's close, and predicting the return from today to tomorrow
Trading 18x is optimal but 5x is conservative


In [150]:
position = pd.DataFrame()
for symbol in symbols:
    capital = leverage * acc_size / len(symbols)
    
    if symbol in base_to_usd.keys():
        lot_usd = lot_size / prc_to_usd[base_to_usd[symbol]].iloc[-1]
    else:
        lot_usd = lot_size * 1
    
    pos_value = ta_features[symbol]['Pos'] * capital

    position[symbol] = pos_value / lot_usd
np.around(position,2)[::-1]

,EURUSD=X,JPY=X,GBPUSD=X,CHF=X,AUDUSD=X,CAD=X,NZDUSD=X
Date,,,,,,,
2024-01-25,-0.26,0.43,0.22,NaN,-0.65,0.29,-0.93
2024-01-24,-0.39,0.14,-0.45,-0.00,0.22,-0.71,-0.70
2024-01-23,-0.00,0.14,-0.45,0.14,-0.00,-0.57,1.17
2024-01-22,-0.66,0.14,0.34,-0.43,-0.43,0.14,-0.47
2024-01-19,0.39,0.29,0.11,-0.57,-0.65,0.00,-0.23
...,...,...,...,...,...,...,...
2023-02-14,-0.39,0.14,0.34,0.71,0.22,0.43,-0.70
2023-02-13,0.66,-0.29,0.00,0.14,1.08,0.29,1.17
2023-02-10,0.66,-0.29,0.56,0.00,1.08,-0.29,1.17


In [152]:
position_change = np.around(position,2).diff()
position_change[::-1]

,EURUSD=X,JPY=X,GBPUSD=X,CHF=X,AUDUSD=X,CAD=X,NZDUSD=X
Date,,,,,,,
2024-01-25,0.13,0.29,0.67,NaN,-0.87,1.00,-0.23
2024-01-24,-0.39,0.00,0.00,-0.14,0.22,-0.14,-1.87
2024-01-23,0.66,0.00,-0.79,0.57,0.43,-0.71,1.64
2024-01-22,-1.05,-0.15,0.23,0.14,0.22,0.14,-0.24
2024-01-19,0.65,0.43,-0.23,-0.71,-1.73,0.00,-1.40
...,...,...,...,...,...,...,...
2023-02-14,-1.05,0.43,0.34,0.57,-0.86,0.14,-1.87
2023-02-13,0.00,0.00,-0.56,0.14,0.00,0.58,0.00
2023-02-10,0.66,-1.00,0.00,-0.71,0.00,-0.29,0.00
